In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
df.head()

### Check Data for Null values

In [ ]:
sns.heatmap(df.isnull(),cmap='viridis',cbar=False)

### No Null values in our Data Frame

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(),annot=True)

### Since all our feature is numeric. Lets check how many of them are discrete and how many are contineous features

In [ ]:
discrete_feature = [feature for feature in df.columns if len(df[feature].unique()) < 25]

In [ ]:
print(discrete_feature)
print('Total Discrete feature :',len(discrete_feature))

In [ ]:
for feature in discrete_feature:
    data = df.copy()
    
    data.groupby(feature)['DEATH_EVENT'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('DEATH_EVENT')
    plt.title(feature)
    plt.show()

In [ ]:
sns.countplot(x='anaemia',hue='DEATH_EVENT',data=df)

### People with low anaemia are more prone to death event

In [ ]:
sns.countplot(x='diabetes',hue='DEATH_EVENT',data=df)

In [ ]:
sns.catplot(x='ejection_fraction',hue='DEATH_EVENT',data=df,kind='count');

In [ ]:
sns.catplot(x='high_blood_pressure',hue='DEATH_EVENT',data=df,kind='count');

In [ ]:
sns.countplot(x='sex',hue='DEATH_EVENT',data=df)

In [ ]:
sns.countplot(x='smoking',data=df,hue='DEATH_EVENT')

In [ ]:
contineous_feature = [feature for feature in df.columns if feature not in discrete_feature]

In [ ]:
print(contineous_feature)
print('Total contineous feature count is  : ',len(contineous_feature))

In [ ]:
df.head()

In [ ]:
for feature in contineous_feature:
    data = df.copy()
    #data.groupby(feature)['DEATH_EVENT'].median().plot.bar()
    plt.scatter(data[feature],data['DEATH_EVENT'])
    plt.xlabel(feature)
    plt.ylabel('DEATH_EVENT')
    plt.title(feature)
    plt.show()

In [ ]:
X_scale  = df.drop('DEATH_EVENT',axis=1)
y_scale = df['DEATH_EVENT']

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
sc = StandardScaler()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scale, y_scale, test_size=0.3, random_state=0)

In [ ]:
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

In [ ]:
feature_for_model = SelectFromModel(Lasso(alpha=0.05,random_state=0))
feature_for_model.fit(X_train_scaled,y_train)

In [ ]:
feature_for_model.get_support()

In [ ]:
X.columns

In [ ]:
#mask = feature_for_model.get_support()
#X_train = X_train.columns[mask]

In [ ]:
X = X[['age','ejection_fraction','serum_creatinine','serum_sodium','time']]
y = df['DEATH_EVENT']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scale, y_scale, test_size=0.3, random_state=0)

In [ ]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print(confusion_matrix(y_test,y_pred))

In [ ]:
 #Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]
# Method of selecting samples for training each tree
# bootstrap = [True, False]


In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestClassifier()

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(X_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
y_pred = rf_random.predict(X_test)

In [ ]:
print(confusion_matrix(y_pred,y_test))

In [ ]:
print(classification_report(y_pred,y_test))

In [ ]:
rf_random.best_params_

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))